In [6]:
import sys
sys.path.append('/workspace/shared_workspace/deep-learning-models/models/vision/detection/')
import tensorflow as tf
import horovod.tensorflow as hvd
hvd.init()

In [39]:
from awsdet.datasets.coco import CocoDataset
from awsdet.datasets.data_generator import DataGenerator
from awsdet.datasets.loader.build_loader import build_dataloader

from awsdet.models.backbones import keras_backbone
from awsdet.models.necks import fpn
from awsdet.models.anchor_heads.rpn_head import RPNHead
from awsdet.core.bbox.bbox_target import ProposalTarget
from awsdet.models.roi_extractors.roi_align import PyramidROIAlign

In [8]:
dataset_dir = '/workspace/shared_workspace/data/coco/coco/'
subset = 'train'
coco_dataset = CocoDataset(dataset_dir, subset, train=True)

loading annotations into memory...
Done (t=14.16s)
creating index...
index created!


In [13]:
coco_tdf = build_dataloader(coco_dataset, 2)[0]

In [40]:
backbone = keras_backbone.KerasBackbone('ResNet50V1')
neck = fpn.FPN()
rpn_head = RPNHead()
bbox_target = ProposalTarget()
bbox_roi_extractor = PyramidROIAlign(pool_shape=[7, 7], pool_type='avg')

In [36]:
img, img_metas, gt_boxes, gt_class_ids = next(iter(coco_tdf))

Starting new loop for GPU: 0


In [41]:
training=True
C2, C3, C4, C5 = backbone(img, training=training)
P2, P3, P4, P5, P6 = neck((C2, C3, C4, C5), training=training)
rpn_feature_maps = [P2, P3, P4, P5, P6]
rcnn_feature_maps = [P2, P3, P4, P5]
rpn_class_logits, rpn_probs, rpn_deltas = rpn_head(rpn_feature_maps, training=training)
proposals_list = rpn_head.get_proposals(
            rpn_probs, rpn_deltas, img_metas, training=training)
if training:
    rois_list, rcnn_target_matchs, rcnn_target_deltas, inside_weights, outside_weights = \
        bbox_target.build_targets(proposals_list, gt_boxes, gt_class_ids, img_metas)
else:
    rois_list = proposals_list
pooled_regions_list = bbox_roi_extractor(
    (rois_list, rcnn_feature_maps, img_metas), training=training)

In [46]:
pooled_regions_list[0]

<tf.Tensor: shape=(512, 7, 7, 256), dtype=float32, numpy=
array([[[[ 2.89023781e+00,  1.28407879e+01, -7.24552155e+00, ...,
          -7.62648010e+00, -2.95429802e+00,  1.61181641e+01],
         [ 3.82303500e+00,  2.31146984e+01, -7.15511847e+00, ...,
          -8.97478485e+00, -5.69383335e+00,  1.97338066e+01],
         [ 5.18974972e+00,  2.85168152e+01, -7.87016106e+00, ...,
          -9.77050400e+00, -6.81672955e+00,  2.05449753e+01],
         ...,
         [ 6.92493773e+00,  2.86818409e+01, -5.07865715e+00, ...,
          -9.98560333e+00, -7.85034084e+00,  1.63385696e+01],
         [ 6.58075190e+00,  2.69700012e+01,  4.77733165e-01, ...,
          -1.05860538e+01, -1.30891218e+01,  1.17802677e+01],
         [ 2.92579174e-01,  1.89024487e+01, -2.63419676e+00, ...,
          -1.04240341e+01, -1.15734158e+01,  8.09477997e+00]],

        [[-8.76183808e-01,  7.72663450e+00, -1.72093844e+00, ...,
          -4.14033031e+00, -3.06324244e+00,  6.48612881e+00],
         [-1.37619412e+00,  1.